# QRCodeStaticCode Maker

@alessioborgi

QRStaticCode allows you to creating static, unlimited-use QR codes with custom styles, embedded icons, and optional captions.


### 1: IMPORTING LIBRARIES

In [312]:
import sys
!{sys.executable} -m pip install "qrcode[pil]" Pillow


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
from qrcode import QRCode
from qrcode.constants import ERROR_CORRECT_H
from qrcode.image.styledpil import StyledPilImage
from qrcode.image.styles.moduledrawers import RoundedModuleDrawer
from qrcode.image.styles.colormasks import SolidFillColorMask
from PIL import Image, ImageDraw, ImageFont, ImageColor
import os, PIL

### 2: FUNCTIONS DEFINITION

In [ ]:
def generate_stylish_qr(
    url: str,                                       # URL to encode.     
    output_path: str = "qr_with_text.png",          # output file.
    box_size: int = 20,
    border: int = 4,
    fill_color: str = "#1f2937",
    back_color: str = "white",
    icon_path: str = None,
    icon_color: str = None,
    icon_size_ratio: float = 0.25,
    icon_border: int = 30,
    caption: str = None,
    font_path: str = None,
    font_size: int = None,
    caption_size_ratio: float = 0.30,   # 30% of QR width
    caption_color: str = "#1f2937",
    caption_padding: int = 4,
    caption_offset: int = -20            # negative = into QR’s bottom border
):
    # normalize colors to (R,G,B)
    def norm_rgb(c):
        if isinstance(c, (tuple,list)) and len(c)==1: c=c[0]
        return ImageColor.getrgb(c) if isinstance(c,str) else c

    fc, bc = norm_rgb(fill_color), norm_rgb(back_color)
    ic, cc = norm_rgb(icon_color or fill_color), norm_rgb(caption_color)

    # 1) Build QR
    qr = QRCode(error_correction=ERROR_CORRECT_H, box_size=box_size, border=border)
    qr.add_data(url)
    qr.make(fit=True)

    # 2) Render modules
    qr_img = qr.make_image(
        image_factory=StyledPilImage,
        module_drawer=RoundedModuleDrawer(),
        color_mask=SolidFillColorMask(front_color=fc, back_color=bc)
    ).convert("RGBA")
    qr_w, qr_h = qr_img.size

    # 3) Overlay tinted icon silhouette (unchanged)
    if icon_path:
        icon = Image.open(icon_path).convert("RGBA")
        mask = icon.convert("L").point(lambda x:255 if x<250 else 0)
        r,g,b = ic
        tinted = Image.new("RGBA", icon.size, (r,g,b,255))
        tinted.putalpha(mask)

        icon_px = int(qr_w * icon_size_ratio)
        tinted = tinted.resize((icon_px, icon_px), Image.Resampling.LANCZOS)

        draw_qr = ImageDraw.Draw(qr_img)
        cx, cy = qr_w//2, qr_h//2
        rad = icon_px//2 + icon_border
        draw_qr.ellipse([cx-rad, cy-rad, cx+rad, cy+rad], fill=bc)

        qr_img.paste(tinted, (cx-icon_px//2, cy-icon_px//2), tinted)

    # 4) If no caption, done.
    if not caption:
        qr_img.save(output_path)
        print(f"Saved QR → {output_path}")
        return

    # 5) Compute font_size from ratio if not given
    if font_size is None:
        font_size = max(12, int(qr_w * caption_size_ratio))

    # 6) Load a real TTF (user’s or PIL’s DejaVuSans), else fallback
    font = None
    if font_path:
        try: font = ImageFont.truetype(font_path, font_size)
        except: font = None
    if font is None:
        try:
            pil_fonts = os.path.join(os.path.dirname(PIL.__file__), "Fonts")
            font = ImageFont.truetype(os.path.join(pil_fonts, "DejaVuSans.ttf"), font_size)
        except:
            font = ImageFont.load_default()

    # 7) Measure caption
    drawer = ImageDraw.Draw(qr_img)
    x0,y0,x1,y1 = drawer.textbbox((0,0), caption, font=font)
    text_w, text_h = x1-x0, y1-y0

    # 8) Create canvas just tall enough
    final_h = qr_h + caption_padding + text_h
    canvas = Image.new("RGBA", (qr_w, final_h), bc)
    canvas.paste(qr_img, (0,0))

    # 9) Draw caption at true offset
    draw = ImageDraw.Draw(canvas)
    tx = (qr_w - text_w)//2
    ty = qr_h + caption_padding + caption_offset
    draw.text((tx, ty), caption, font=font, fill=cc)

    # 10) Save
    canvas.save(output_path)
    print(f"Saved QR + caption → {output_path}")


### 3: MAIN (EXAMPLE USAGE)

In [315]:
# Main parameters.
# URL to encode, output path, and colors.
url = "https://alessioborgi.github.io/Z-SASLM.github.io/"          # URL to encode.                    
output_path = "./results/Z-SASLM_QR_Code.png"                      # Output file path.
fill_color = "#1f2937"                                           # QR module color.   
icon_path = "./icons/home.png"                                     # Path to icon image (optional).
caption = "HomePage"                                               # Caption text (optional).                  
caption_color = "#1f2937"                                        # Caption text color. 


# Other optional parameters (here they are set as default).
box_size = 20                           # Size of each QR module in pixels. 
border = 4                              # Size of the border (minimum is 4).  
back_color = "white"                    # Background color.    
icon_color = None                       # Tint color for icon (defaults to fill_color).
icon_size_ratio = 0.25                  # Icon size as a fraction of QR size.
icon_border = 30                        # Space between icon and QR modules.
font_size = 2000                        # Font size for caption.    
caption_padding = 2                     # Space between QR and text.
caption_size_ratio=0.10                 # Caption size as a fraction of QR size.
caption_offset=-70                      # Offset for caption position (positive = down, negative = up).
font_path="/Library/Fonts/Ayuthaya.ttf" # Path to font file (optional).


In [316]:
if __name__ == "__main__":
    generate_stylish_qr(
        url=url,
        output_path=output_path,
        fill_color=fill_color,
        back_color=back_color,
        icon_path=icon_path,
        icon_size_ratio=icon_size_ratio,
        icon_border=icon_border,
        caption=caption,
        font_path=font_path,
        caption_size_ratio=caption_size_ratio,        
        caption_padding=caption_padding,        
        caption_offset=caption_offset           
    )


Saved QR + caption → ./results/Z-SASLM_QR_Code.png
